In [1]:
import os,sys
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *
from tqdm import tqdm
from collections import Counter
import pandas as pd
import shutil
import pickle
import gc, math
from math import log, floor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import *
from scipy import sparse
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
max_words = 23 #dict size (tri-gram總數量)
max_length = 115 #第一轉折點 tri-gram num (最靠近outlier數字的斜率變化轉折點) 超過此數量的hkl拿掉
min_length = 30 #最小的轉折點(第三個轉折點) #Q1

In [3]:
root_dir = "./data/Hooklog/MikeGithub/"
fam_dirs = next(os.walk(root_dir))[1]
trainX = []
trainY = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
#     hl_list = list(filter(lambda f: f.endswith(".enc.npy"), hl_list)) # in case some non-hooklog file in the folder
    hl_list = list(filter(lambda f: f.endswith(".single.npy"), hl_list)) #load norm data
    for npy in hl_list:
        tri_enc = np.load(npy)
        if len(tri_enc)>max_length:
            tri_enc = tri_enc[:max_length]
        if len(tri_enc)<min_length:
            continue
        trainY.append(fam)
        trainX.append(tri_enc)
print(len(trainY),len(trainX))

100%|██████████| 13/13 [00:00<00:00, 121.04it/s]

486 486


In [4]:
fam_code = {}
for i,v in enumerate(list(set(trainY))):
    fam_code[v] = i
# fam_code = pickle.load(file=open(root_dir+"fam_dict_enc.pkl",'rb'))
train_Y = []
for fam in trainY:
    train_Y.append(fam_code[fam])
train_Y = np.array(train_Y)
pickle.dump(obj=fam_code,file=open(root_dir+"fam_dict_enc_single_Mike.pkl",'wb'))
train_Y

array([ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3

In [5]:
#encode to int (remember the encoding)
train_X = tf.keras.preprocessing.sequence.pad_sequences(trainX,maxlen=max_length,padding='post',value=0,dtype='int32')
print(train_X.shape, train_Y.shape)

(486, 115) (486,)


In [6]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [14]:
X_train, y_train, X_test, y_test = split_valid_set(train_X, train_Y, 0.9)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
assert len(set(y_test)) == len(set(y_train))
x = dict(Counter(y_test))
temp = {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
assert temp[list(temp.keys())[0]] > 1
print(set(y_test))
# if assertion error, run more time for this cell

(437, 115) (437,) (49, 115) (49,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


# Build Model

In [15]:
emb_dim = 128#validXr.shape[-1] 
max_length = X_test.shape[-1] # max sequence length
cat_num = y_test.shape[0]
do = 0.1
vocabulary_size = max_words+1
hidden_dim = 64
EPOCHS=2000
# traina = True #改
batch_size = 128 #改

In [256]:
#從0開始
# init_w = tensorflow.keras.initializers.RandomNormal(mean=0.9,stddev=0.06)
# init_b = tensorflow.keras.initializers.RandomNormal(mean=0.7,stddev=0.2)
#從1開始
init_w = tensorflow.keras.initializers.RandomNormal(mean=0.5,stddev=0.5) #mean=1.667,stddev=0.333
init_b = tensorflow.keras.initializers.RandomNormal(mean=0.5,stddev=0.5) #mean=1.,stddev=0.5
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0],reshuffle_each_iteration=True).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)



In [257]:
class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn1 = LSTM(emb_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        #x = self.att([x1,x1])
        x = self.rnn1(x1)
        x = self.bn1(x)
        x = self.att([x,x1,x1])
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
# model2 = base_model_1()
# saveP1 = './model/Hooklog/2020020201/model1' #LSTM+bn+att
# model2.load_weights(saveP1)
# model1.layers[0].set_weights(model2.layers[0].get_weights())
# model1.layers[1].set_weights(model2.layers[1].get_weights())
# model1.layers[2].set_weights(model2.layers[2].get_weights())
# model1.layers[3].set_weights(model2.layers[3].get_weights())
# model1 = tf.saved_model.load(saveP1+'_all')
model1(X_train[:2])

(<tf.Tensor: shape=(2, 115, 128), dtype=float32, numpy=
 array([[[-0.0341964 ,  0.00026212,  0.02864249, ...,  0.01576651,
           0.03652966,  0.0476935 ],
         [ 0.02363114,  0.01286485, -0.03548645, ...,  0.02696906,
          -0.03813945, -0.04309798],
         [-0.01419364,  0.01937393,  0.04240689, ..., -0.04561083,
          -0.03324284,  0.02403304],
         ...,
         [ 0.02363114,  0.01286485, -0.03548645, ...,  0.02696906,
          -0.03813945, -0.04309798],
         [-0.03680889,  0.03981799,  0.03626403, ..., -0.01590227,
          -0.01725348,  0.01093619],
         [-0.0341964 ,  0.00026212,  0.02864249, ...,  0.01576651,
           0.03652966,  0.0476935 ]],
 
        [[ 0.02363114,  0.01286485, -0.03548645, ...,  0.02696906,
          -0.03813945, -0.04309798],
         [ 0.02363114,  0.01286485, -0.03548645, ...,  0.02696906,
          -0.03813945, -0.04309798],
         [ 0.02363114,  0.01286485, -0.03548645, ...,  0.02696906,
          -0.03813945, -0.04

In [258]:
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        #self.rnn2 = Bidirectional(LSTM(int(hidden_dim),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/4))
        self.bn2 = BatchNormalization(name='bn2')
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        #x = self.bn2(x2)
        #x = self.dn(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [259]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: shape=(1, 115, 114), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [260]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def word_reproduce1(prediction_int):
    return prediction_int
def word_reproduce2(prediction_int):
    return prediction_int
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

# optimizer1 = tf.keras.optimizers.SGD(learning_rate=0.0044, momentum=0.5, nesterov=True)
optimizer1 = tf.keras.optimizers.Nadam()#Adam(learning_rate=0.0003)

In [261]:
train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [262]:
alpha = 0.00000005#0.000001#0.00000001#0.000002 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
# 1- => alpha: 0.0000001移動太大，init不可以全0
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
DateID = '2020020701'
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = 1-loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

    
# word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
# word_idx_pd['ori_word'] = word_idx_pd.index 
# word_idx_pd = word_idx_pd.set_index(["ID"])
# print(word_idx_pd)

ep = -1
@tf.function(experimental_relax_shapes=True)
def test_step_inf(x,yc,epoch):
    #ori_text = getKeysByValues(list(x))
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    #if ep != epoch:
        #ep = epoch
        #x = tf.dtypes.cast(x,dtype=tf.float32)
    pred_imp3_ = tf.squeeze(pred_imp2)
    pred_imp3 = tf.dtypes.cast(pred_imp3_,dtype=tf.int32)
    used_x = tf.math.multiply(x,pred_imp3)
    filter_emb = tf.math.multiply(emb,pred_imp2)
        #word_reproduce1(x)
        #word_reproduce2(used_x)
        #used_text = getKeysByValues(list(used_x))
        #tf.print(used_x)
        #print(emb.numpy(),pred_imp, x,pred_imp3,used_x)
        #int_ori_text.append(x)
        #int_fil_text.append(used_x)
    
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    return x, used_x, pred_cat, emb, filter_emb

In [263]:
EPOCHS = 2000


saveP1 = './model/Hooklog/'+DateID+'/model1'
saveP2 = './model/Hooklog/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

int_ori_text =[]
int_fil_text = []
int_label = []
pred_label = []
ori_emb = []
fil_emb = []
epoch_num = []
fam_lab = []
pred_lab = []
# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    epoch = tf.convert_to_tensor(epoch, dtype=tf.int64)
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        ori,fil,pred_cat,emb,filter_emb = test_step_inf(test_text, test_labels,epoch) #with restore words
        ori_emb.append(emb.numpy()) #一個epoch有很多個batches
        fil_emb.append(filter_emb.numpy()) #一個epoch有很多個batches
        epoch_num.append(epoch.numpy()) #一個epoch一個數字
        fam_lab.append(test_labels.numpy())#一個epoch有很多個batches
        pred_lab.append(pred_cat.numpy())#一個epoch有很多個batches
        #print(k2)
        #test_step(test_text, test_labels) #original
    int_ori_text.append(ori.numpy())
    int_fil_text.append(fil.numpy())
    int_label.append(test_labels.numpy()) #true label
    pred_label.append(pred_cat.numpy())

    
    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))
    #int_ori_text.append(word_reproduce1.result().numpy())
    #int_fil_text.append(word_reproduce2.result().numpy())
    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<test_accuracy.result()*100:
        tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
#         model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    #word_reproduce1.reset_states()
    #word_reproduce2.reset_states()
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()
pred_lab2 = []
for pred in pred_lab:
    pred_lab2.append(list(np.argmax(pred,axis=-1)))

Epoch 1, Total Loss: 2.4841270446777344, Clf Loss: 2.4841270446777344, Filter Loss: 0.5851390361785889, Seq Loss: 1.0, Accuracy Rate:  8.92%, Ones Portion: 0.0,             Test_Total_Loss: 2.4837050437927246, Test_Clf_Loss: 2.4837050437927246, Test_Filter_Loss: 0.41870978474617004, TEST_Seq_Loss: 1.0, Test_Accuracy_Rate: 16.33%, Test_Ones_Portion: 0.0
INFO:tensorflow:Assets written to: ./model/Hooklog/2020020701/model1_all/assets
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020701/model1 ./model/Hooklog/2020020701/model2
Epoch 2, Total Loss: 2.481487989425659, Clf Loss: 2.481487989425659, Filter Loss: 0.5748826265335083, Seq Loss: 1.0, Accuracy Rate: 10.98%, Ones Portion: 0.0,             Test_Total_Loss: 2.4824373722076416, Test_Clf_Loss: 2.4824373722076416, Test_Filter_Loss: 0.43138542771339417, TEST_Seq_Loss: 1.0, Test_Accuracy_Rate: 16.33%, Test_Ones_Portion: 0.0
Epoch 3, Total Loss: 2.4789764881134033, Clf Loss: 2.4789764881134033, Filter Loss: 0.5592901706695557, Seq Loss: 1.0,

Epoch 20, Total Loss: 1.5350282192230225, Clf Loss: 1.5350282192230225, Filter Loss: 0.47385483980178833, Seq Loss: 0.33597636222839355, Accuracy Rate: 46.45%, Ones Portion: 0.6640259027481079,             Test_Total_Loss: 1.6281927824020386, Test_Clf_Loss: 1.6281927824020386, Test_Filter_Loss: 0.5125779509544373, TEST_Seq_Loss: 0.39858394861221313, Test_Accuracy_Rate: 38.78%, Test_Ones_Portion: 0.6014160513877869
Epoch 21, Total Loss: 1.4728851318359375, Clf Loss: 1.4728851318359375, Filter Loss: 0.4722582697868347, Seq Loss: 0.33006012439727783, Accuracy Rate: 46.68%, Ones Portion: 0.6699398756027222,             Test_Total_Loss: 1.5954344272613525, Test_Clf_Loss: 1.5954344272613525, Test_Filter_Loss: 0.5156093239784241, TEST_Seq_Loss: 0.39858394861221313, Test_Accuracy_Rate: 38.78%, Test_Ones_Portion: 0.6014160513877869
Epoch 22, Total Loss: 1.4782681465148926, Clf Loss: 1.4782681465148926, Filter Loss: 0.47669029235839844, Seq Loss: 0.3700708746910095, Accuracy Rate: 48.74%, Ones P

Epoch 39, Total Loss: 0.9335623383522034, Clf Loss: 0.9335623383522034, Filter Loss: 0.4401910901069641, Seq Loss: 0.31961512565612793, Accuracy Rate: 76.43%, Ones Portion: 0.6803986430168152,             Test_Total_Loss: 1.1328791379928589, Test_Clf_Loss: 1.1328791379928589, Test_Filter_Loss: 0.5491113066673279, TEST_Seq_Loss: 0.39858394861221313, Test_Accuracy_Rate: 63.27%, Test_Ones_Portion: 0.6014160513877869
Epoch 40, Total Loss: 0.9423735737800598, Clf Loss: 0.942373514175415, Filter Loss: 0.43953660130500793, Seq Loss: 0.31948262453079224, Accuracy Rate: 76.20%, Ones Portion: 0.6805996894836426,             Test_Total_Loss: 1.2122821807861328, Test_Clf_Loss: 1.2122821807861328, Test_Filter_Loss: 0.5500576496124268, TEST_Seq_Loss: 0.3768354654312134, Test_Accuracy_Rate: 57.14%, Test_Ones_Portion: 0.6247591376304626
Epoch 41, Total Loss: 0.9689632654190063, Clf Loss: 0.9689632058143616, Filter Loss: 0.4414970874786377, Seq Loss: 0.3258262276649475, Accuracy Rate: 72.31%, Ones Port

Epoch 59, Total Loss: 0.7563637495040894, Clf Loss: 0.7563637495040894, Filter Loss: 0.43940895795822144, Seq Loss: 0.3187396824359894, Accuracy Rate: 79.41%, Ones Portion: 0.6817951202392578,             Test_Total_Loss: 0.9245206713676453, Test_Clf_Loss: 0.9245206117630005, Test_Filter_Loss: 0.5537073016166687, TEST_Seq_Loss: 0.3355060815811157, Test_Accuracy_Rate: 69.39%, Test_Ones_Portion: 0.6655200719833374
Epoch 60, Total Loss: 0.694493293762207, Clf Loss: 0.6944931745529175, Filter Loss: 0.43624693155288696, Seq Loss: 0.29795950651168823, Accuracy Rate: 79.41%, Ones Portion: 0.7025301456451416,             Test_Total_Loss: 0.9160177707672119, Test_Clf_Loss: 0.9160177111625671, Test_Filter_Loss: 0.5540702939033508, TEST_Seq_Loss: 0.33215779066085815, Test_Accuracy_Rate: 69.39%, Test_Ones_Portion: 0.6684187054634094
Epoch 61, Total Loss: 0.7333788871765137, Clf Loss: 0.7333788871765137, Filter Loss: 0.4387993812561035, Seq Loss: 0.3133121728897095, Accuracy Rate: 79.41%, Ones Port

INFO:tensorflow:Assets written to: ./model/Hooklog/2020020701/model1_all/assets
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2020020701/model1 ./model/Hooklog/2020020701/model2
Epoch 79, Total Loss: 0.6346321105957031, Clf Loss: 0.6346319913864136, Filter Loss: 0.43236368894577026, Seq Loss: 0.2394055873155594, Accuracy Rate: 81.69%, Ones Portion: 0.7611793875694275,             Test_Total_Loss: 0.8208232522010803, Test_Clf_Loss: 0.8208231925964355, Test_Filter_Loss: 0.5576136112213135, TEST_Seq_Loss: 0.28518444299697876, Test_Accuracy_Rate: 75.51%, Test_Ones_Portion: 0.7150525450706482
Epoch 80, Total Loss: 0.6466478109359741, Clf Loss: 0.6466477513313293, Filter Loss: 0.4347853660583496, Seq Loss: 0.2537887692451477, Accuracy Rate: 81.69%, Ones Portion: 0.7467852234840393,             Test_Total_Loss: 0.9030910134315491, Test_Clf_Loss: 0.9030909538269043, Test_Filter_Loss: 0.5578593611717224, TEST_Seq_Loss: 0.27762967348098755, Test_Accuracy_Rate: 75.51%, Test_Ones_Portion: 0.7225754261

...

Epoch 1905, Total Loss: 0.056430622935295105, Clf Loss: 0.05643058568239212, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24442946910858154, Test_Clf_Loss: 0.24442940950393677, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1906, Total Loss: 0.04622867703437805, Clf Loss: 0.04622863605618477, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2452870011329651, Test_Clf_Loss: 0.2452869415283203, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1907, Total Loss: 0.045899007469415665, Clf Loss: 0.04589896649122238, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24562674760818481, Test_Clf_Loss: 0.24562668800354004, Test_Filter_Loss: 0.599999

Epoch 1928, Total Loss: 0.05903855711221695, Clf Loss: 0.059038519859313965, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2454545497894287, Test_Clf_Loss: 0.24545449018478394, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1929, Total Loss: 0.042991720139980316, Clf Loss: 0.04299167916178703, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24602541327476501, Test_Clf_Loss: 0.24602535367012024, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1930, Total Loss: 0.04321988672018051, Clf Loss: 0.04321984201669693, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24641019105911255, Test_Clf_Loss: 0.24641013145446777, Test_Filter_Loss: 0.59999

Epoch 1951, Total Loss: 0.04339784383773804, Clf Loss: 0.04339780658483505, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24309641122817993, Test_Clf_Loss: 0.24309635162353516, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1952, Total Loss: 0.05536536127328873, Clf Loss: 0.05536532402038574, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2426157146692276, Test_Clf_Loss: 0.24261565506458282, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1953, Total Loss: 0.04358895495533943, Clf Loss: 0.04358891770243645, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2423100620508194, Test_Clf_Loss: 0.24231000244617462, Test_Filter_Loss: 0.59999996

Epoch 1974, Total Loss: 0.042049940675497055, Clf Loss: 0.04204989969730377, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2465275377035141, Test_Clf_Loss: 0.24652747809886932, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1975, Total Loss: 0.041076987981796265, Clf Loss: 0.04107695072889328, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.2467164248228073, Test_Clf_Loss: 0.24671636521816254, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1976, Total Loss: 0.0438675656914711, Clf Loss: 0.04386752471327782, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.246323823928833, Test_Clf_Loss: 0.24632376432418823, Test_Filter_Loss: 0.599999964

Epoch 1997, Total Loss: 0.04162611812353134, Clf Loss: 0.04162607714533806, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24571457505226135, Test_Clf_Loss: 0.24571451544761658, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1998, Total Loss: 0.04441584274172783, Clf Loss: 0.044415801763534546, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24546176195144653, Test_Clf_Loss: 0.24546170234680176, Test_Filter_Loss: 0.5999999642372131, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 93.88%, Test_Ones_Portion: 1.0
Epoch 1999, Total Loss: 0.04026276618242264, Clf Loss: 0.04026272892951965, Filter Loss: 0.3999999761581421, Seq Loss: 0.0, Accuracy Rate: 99.08%, Ones Portion: 1.0,             Test_Total_Loss: 0.24523574113845825, Test_Clf_Loss: 0.24523568153381348, Test_Filter_Loss: 0.59999

In [9]:
"""
class test_model(Model):
    def __init__(self):
        super(test_model, self).__init__()
        self.emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn = LSTM(hidden_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.bn2 = BatchNormalization(name='bn2')
        self.att = Attention(name='selfatt')
        self.rnn2 = GRU(int(hidden_dim/2),name='common_extract2'
                      ,trainable=True)
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        
        self.cnn = Conv1D(int(hidden_dim/2),strides=2,kernel_size=4)
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
        #self.out = Dense(max(train_Y)+1,activation='softmax',name='clf')
        self.flat = Flatten()
    def call(self,x):
        x1 = self.emb(x)
        #x = self.cnn(x)
        x = self.att([x1,x1,x1])
        #x = self.rnn(x)
        #x = self.bn1(x)
        #x = self.att([x,x1,x1])
        #x = self.att([x,x,x])
        x = self.rnn2(x)
        x = self.bn2(x)
        #x = self.flat(x)
        x = self.dn(x)
        y2 = self.out(x)
        return y2
model = test_model()
# batch_size = 32
try:
    train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0],reshuffle_each_iteration=True).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)
except NameError:
    train_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).shuffle(train_X.shape[0]).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).batch(batch_size)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Nadam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') 
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x,yc):
    with tf.GradientTape() as tape:
        pred_cat = model(x) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss = loss_object(yc, pred_cat)
    trainable_variable = model.trainable_variables
    gradients = tape.gradient(loss,trainable_variable)
    optimizer.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_accuracy(yc, pred_cat) #acc_rate

@tf.function
def test_step(x,yc):
    pred_cat = model(x) #weight: pred_imp ; binary:pred_imp2
    t_loss = loss_object2(yc, pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)


for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels) #with restore words
    
    template = 'Epoch {}, Total Loss: {}, Accuracy Rate: {:5.2f}%,\
            Test_Total_Loss: {}, Test_Accuracy_Rate: {:5.2f}%'
    print(template.format(epoch+1,train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100
                        ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
"""    

Epoch 1, Total Loss: 2.4634788036346436, Accuracy Rate: 16.02%,            Test_Total_Loss: 2.4256539344787598, Test_Accuracy_Rate: 34.69%
Epoch 2, Total Loss: 2.423973560333252, Accuracy Rate: 27.46%,            Test_Total_Loss: 2.3692984580993652, Test_Accuracy_Rate: 44.90%
Epoch 3, Total Loss: 2.374782085418701, Accuracy Rate: 31.58%,            Test_Total_Loss: 2.2945475578308105, Test_Accuracy_Rate: 44.90%
Epoch 4, Total Loss: 2.3063619136810303, Accuracy Rate: 33.87%,            Test_Total_Loss: 2.194746255874634, Test_Accuracy_Rate: 44.90%
Epoch 5, Total Loss: 2.2172152996063232, Accuracy Rate: 33.87%,            Test_Total_Loss: 2.0625734329223633, Test_Accuracy_Rate: 44.90%
Epoch 6, Total Loss: 2.100715160369873, Accuracy Rate: 33.87%,            Test_Total_Loss: 1.9044348001480103, Test_Accuracy_Rate: 44.90%
Epoch 7, Total Loss: 1.988112211227417, Accuracy Rate: 33.87%,            Test_Total_Loss: 1.773451805114746, Test_Accuracy_Rate: 44.90%
Epoch 8, Total Loss: 1.8965494632

Epoch 60, Total Loss: 0.315942645072937, Accuracy Rate: 92.68%,            Test_Total_Loss: 0.3443942666053772, Test_Accuracy_Rate: 89.80%
Epoch 61, Total Loss: 0.3571322560310364, Accuracy Rate: 92.91%,            Test_Total_Loss: 0.3495045006275177, Test_Accuracy_Rate: 91.84%
Epoch 62, Total Loss: 0.2994227409362793, Accuracy Rate: 94.28%,            Test_Total_Loss: 0.32165268063545227, Test_Accuracy_Rate: 89.80%
Epoch 63, Total Loss: 0.3033389449119568, Accuracy Rate: 94.05%,            Test_Total_Loss: 0.33033356070518494, Test_Accuracy_Rate: 91.84%
Epoch 64, Total Loss: 0.2724860906600952, Accuracy Rate: 95.42%,            Test_Total_Loss: 0.3141513168811798, Test_Accuracy_Rate: 93.88%
Epoch 65, Total Loss: 0.25926631689071655, Accuracy Rate: 95.88%,            Test_Total_Loss: 0.305127888917923, Test_Accuracy_Rate: 93.88%
Epoch 66, Total Loss: 0.2555406987667084, Accuracy Rate: 96.11%,            Test_Total_Loss: 0.29319852590560913, Test_Accuracy_Rate: 93.88%
Epoch 67, Total Lo

Epoch 118, Total Loss: 0.11680293083190918, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.16955511271953583, Test_Accuracy_Rate: 97.96%
Epoch 119, Total Loss: 0.10913911461830139, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.1685483455657959, Test_Accuracy_Rate: 97.96%
Epoch 120, Total Loss: 0.11287971585988998, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.16943176090717316, Test_Accuracy_Rate: 97.96%
Epoch 121, Total Loss: 0.12860016524791718, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.17253278195858002, Test_Accuracy_Rate: 97.96%
Epoch 122, Total Loss: 0.11405998468399048, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.16621461510658264, Test_Accuracy_Rate: 97.96%
Epoch 123, Total Loss: 0.10440023243427277, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.16237540543079376, Test_Accuracy_Rate: 97.96%
Epoch 124, Total Loss: 0.1048857644200325, Accuracy Rate: 97.71%,            Test_Total_Loss: 0.1621246486902237, Test_Accuracy_Rate: 97.96%
Ep

Epoch 176, Total Loss: 0.06293867528438568, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.17127573490142822, Test_Accuracy_Rate: 97.96%
Epoch 177, Total Loss: 0.06050989776849747, Accuracy Rate: 98.17%,            Test_Total_Loss: 0.17216740548610687, Test_Accuracy_Rate: 97.96%
Epoch 178, Total Loss: 0.06251807510852814, Accuracy Rate: 97.94%,            Test_Total_Loss: 0.17479440569877625, Test_Accuracy_Rate: 97.96%
Epoch 179, Total Loss: 0.06268482655286789, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.17371796071529388, Test_Accuracy_Rate: 97.96%
Epoch 180, Total Loss: 0.0635482594370842, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.16703957319259644, Test_Accuracy_Rate: 97.96%
Epoch 181, Total Loss: 0.056693386286497116, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.1671418994665146, Test_Accuracy_Rate: 97.96%
Epoch 182, Total Loss: 0.05639089643955231, Accuracy Rate: 98.40%,            Test_Total_Loss: 0.16982212662696838, Test_Accuracy_Rate: 97.96%


Epoch 234, Total Loss: 0.04140513762831688, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.28578561544418335, Test_Accuracy_Rate: 95.92%
Epoch 235, Total Loss: 0.04615527391433716, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.28138813376426697, Test_Accuracy_Rate: 95.92%
Epoch 236, Total Loss: 0.0491412952542305, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.2858261466026306, Test_Accuracy_Rate: 95.92%
Epoch 237, Total Loss: 0.04032721742987633, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.2884337604045868, Test_Accuracy_Rate: 95.92%
Epoch 238, Total Loss: 0.04012273624539375, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.290762335062027, Test_Accuracy_Rate: 95.92%
Epoch 239, Total Loss: 0.04377969354391098, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.2904013693332672, Test_Accuracy_Rate: 95.92%
Epoch 240, Total Loss: 0.043402791023254395, Accuracy Rate: 98.63%,            Test_Total_Loss: 0.29120445251464844, Test_Accuracy_Rate: 95.92%
Epoc

...

Epoch 1736, Total Loss: 0.023253625258803368, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5252882242202759, Test_Accuracy_Rate: 95.92%
Epoch 1737, Total Loss: 0.024926483631134033, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5250384211540222, Test_Accuracy_Rate: 95.92%
Epoch 1738, Total Loss: 0.023281151428818703, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5246756076812744, Test_Accuracy_Rate: 95.92%
Epoch 1739, Total Loss: 0.02721536159515381, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5247055888175964, Test_Accuracy_Rate: 95.92%
Epoch 1740, Total Loss: 0.02350069209933281, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5251055359840393, Test_Accuracy_Rate: 95.92%
Epoch 1741, Total Loss: 0.025487594306468964, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5254871845245361, Test_Accuracy_Rate: 95.92%
Epoch 1742, Total Loss: 0.023361526429653168, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5255414843559265, Test_Accuracy_Rate: 9

Epoch 1794, Total Loss: 0.023704832419753075, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5313947796821594, Test_Accuracy_Rate: 95.92%
Epoch 1795, Total Loss: 0.023497240617871284, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5322796106338501, Test_Accuracy_Rate: 95.92%
Epoch 1796, Total Loss: 0.02481076307594776, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5319979786872864, Test_Accuracy_Rate: 95.92%
Epoch 1797, Total Loss: 0.02338649332523346, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5318813920021057, Test_Accuracy_Rate: 95.92%
Epoch 1798, Total Loss: 0.023532724007964134, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5320343971252441, Test_Accuracy_Rate: 95.92%
Epoch 1799, Total Loss: 0.023208368569612503, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5324548482894897, Test_Accuracy_Rate: 95.92%
Epoch 1800, Total Loss: 0.023271147161722183, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5328341722488403, Test_Accuracy_Rate: 9

Epoch 1852, Total Loss: 0.027349665760993958, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5354619026184082, Test_Accuracy_Rate: 95.92%
Epoch 1853, Total Loss: 0.023584600538015366, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5360878705978394, Test_Accuracy_Rate: 95.92%
Epoch 1854, Total Loss: 0.02647409401834011, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5366904139518738, Test_Accuracy_Rate: 95.92%
Epoch 1855, Total Loss: 0.024028221145272255, Accuracy Rate: 98.86%,            Test_Total_Loss: 0.5370890498161316, Test_Accuracy_Rate: 95.92%
Epoch 1856, Total Loss: 0.023742586374282837, Accuracy Rate: 99.08%,            Test_Total_Loss: 0.5373416543006897, Test_Accuracy_Rate: 95.92%
Epoch 1857, Total Loss: 0.025334985926747322, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5373088717460632, Test_Accuracy_Rate: 95.92%
Epoch 1858, Total Loss: 0.023411661386489868, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5372507572174072, Test_Accuracy_Rate: 

Epoch 1910, Total Loss: 0.03618849813938141, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5421084761619568, Test_Accuracy_Rate: 95.92%
Epoch 1911, Total Loss: 0.02365032583475113, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5433001518249512, Test_Accuracy_Rate: 95.92%
Epoch 1912, Total Loss: 0.03381954878568649, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5422941446304321, Test_Accuracy_Rate: 95.92%
Epoch 1913, Total Loss: 0.032600730657577515, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5397481322288513, Test_Accuracy_Rate: 95.92%
Epoch 1914, Total Loss: 0.02385149896144867, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5385597348213196, Test_Accuracy_Rate: 95.92%
Epoch 1915, Total Loss: 0.024404563009738922, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5387056469917297, Test_Accuracy_Rate: 95.92%
Epoch 1916, Total Loss: 0.0375627763569355, Accuracy Rate: 99.31%,            Test_Total_Loss: 0.5417248606681824, Test_Accuracy_Rate: 95.92

Epoch 1968, Total Loss: 0.5909732580184937, Accuracy Rate: 91.30%,            Test_Total_Loss: 0.3000276982784271, Test_Accuracy_Rate: 97.96%
Epoch 1969, Total Loss: 0.22344714403152466, Accuracy Rate: 97.03%,            Test_Total_Loss: 0.1972883641719818, Test_Accuracy_Rate: 97.96%
Epoch 1970, Total Loss: 0.15495839715003967, Accuracy Rate: 97.25%,            Test_Total_Loss: 0.1568981558084488, Test_Accuracy_Rate: 97.96%
Epoch 1971, Total Loss: 0.12124611437320709, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.1694805920124054, Test_Accuracy_Rate: 97.96%
Epoch 1972, Total Loss: 0.14472034573554993, Accuracy Rate: 97.03%,            Test_Total_Loss: 0.18261964619159698, Test_Accuracy_Rate: 97.96%
Epoch 1973, Total Loss: 0.08271212130784988, Accuracy Rate: 97.48%,            Test_Total_Loss: 0.1943798065185547, Test_Accuracy_Rate: 97.96%
Epoch 1974, Total Loss: 0.06741930544376373, Accuracy Rate: 97.71%,            Test_Total_Loss: 0.20453014969825745, Test_Accuracy_Rate: 97.96

## Emb test
* cosine similarity => sklearn shape: (samplesN, features)

ori_emb
fil_emb
epoch_num
fam_lab
pred_lab2

* euclidean distance for intra-dist
* cosine similarity for inter-dist

In [274]:
recent_ep = -1
col = ['Epoch#','Original_Intra_Similarity_Avg','Filter_Intra_Similarity_Avg',
       'Original_Outra_Similarity','Filter_Outra_Similarity']
sim_df = pd.DataFrame(columns=col)
for ep, o_e, f_e, p_l, t_l in tqdm(zip(epoch_num,ori_emb,fil_emb,pred_lab2,fam_lab)): #take out each epoch
    if ep != recent_ep:
        whole_dict_ori = {}
        whole_dict_fil = {}
#         print('EP:',ep)
        recent_ep = ep
        count =  0
    for oe,fe,pl,tl in zip(o_e, f_e, p_l, t_l): #take out each process/sample in the epoch
        count+=1
        try: #依照每個epoch的true label family分
            temp1 = whole_dict_ori[tl] #put by true label
            temp1.append(oe)
            temp2 = whole_dict_fil[tl]
            temp2.append(fe)
            whole_dict_ori[tl] = temp1 #original embedding
            whole_dict_fil[tl] = temp2 #filter embedding
        except KeyError:
            temp1 = [oe]
            temp2 = [fe]
            whole_dict_ori[tl] = temp1 
            whole_dict_fil[tl] = temp2
        assert len(whole_dict_ori[tl]) == len(whole_dict_fil[tl]) > 0
        assert len(whole_dict_ori) == len(whole_dict_fil) > 0
    whole_dict_ori = {k: v for k, v in sorted(whole_dict_ori.items(), key=lambda item: item[0])}
    whole_dict_fil = {k: v for k, v in sorted(whole_dict_fil.items(), key=lambda item: item[0])}
    in_ori_score_li = []
    in_fil_score_li = []
    out_ori_emb_li = []
    out_fil_emb_li = []
    for k,v in whole_dict_ori.items(): #群內
        if len(v)<2: #某個家族只有一筆test sample
            score = 'NaN'
            continue
        emb = np.array(v)
#         emb2 = np.mean(emb,axis=1)
#         emb2 = np.mean(emb2,axis=0)
        emb = emb.reshape(len(v),-1)
        emb2 = np.mean(emb,axis=0)
        score = cosine_similarity(emb)
        score = np.mean(score)
        score = score - 1/len(v) #+ 1e-6*len(emb)
#         if score <= 1e-5:
#             score = 'NaN'
#             continue
        in_ori_score_li.append(score) #每個家族群內的相似度分數
        out_ori_emb_li.append(emb2)
    for k,v in whole_dict_fil.items(): #FIlter Embedding
        if len(v)<2: #某個家族只有一筆test sample
            score = 'NaN'
            continue
        emb = np.array(v)
        emb = emb.reshape(len(v),-1)
        emb2 = np.mean(emb,axis=0)
        score = cosine_similarity(emb)
        score = np.mean(score)
        score = score - 1/len(v) #+ 1e-6
        out_fil_emb_li.append(emb2)
        if score <= 1e-6:
            score = 'NaN'
            continue
        in_fil_score_li.append(score) #每個家族群內的相似度分數
        #ut_fil_emb_li.append(emb2)
    out_ori_score = cosine_similarity(np.array(out_ori_emb_li))
    out_ori_score = np.mean(out_ori_score)
    out_ori_score = out_ori_score - 1/len(out_ori_emb_li)
    out_fil_score = cosine_similarity(np.array(out_fil_emb_li))
    out_fil_score = np.mean(out_fil_score)
    out_fil_score = out_fil_score - 1/len(out_fil_emb_li)
    temp = pd.Series([str(int(ep)+1),np.mean(in_ori_score_li),np.mean(in_fil_score_li),out_ori_score,out_fil_score],col)
    sim_df = sim_df.append(temp,ignore_index=True)
sim_df

0it [00:00, ?it/s]/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/leoqaz12/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/leoqaz12/.local/lib/python3.7/s

,Epoch#,Original_Intra_Similarity_Avg,Filter_Intra_Similarity_Avg,Original_Outra_Similarity,Filter_Outra_Similarity
0,1,0.571640,NaN,0.294336,-0.083333
1,2,0.571703,NaN,0.294415,-0.083333
2,3,0.571796,NaN,0.294588,-0.083333
3,4,0.575421,NaN,0.305074,-0.083333
4,5,0.581270,0.311835,0.321530,-0.037483
...,...,...,...,...,...
1995,1996,0.546963,0.546963,0.240345,0.240345
1996,1997,0.546964,0.546964,0.240337,0.240337
1997,1998,0.546970,0.546970,0.240348,0.240348
1998,1999,0.546970,0.546970,0.240341,0.240341


      ori1/ori2      fil1 /  fil2
mean:     1.0/0.9757028          1.0/0.027777778

reshape:0.96861625/0.6387043       0.96861625/0.027777784
* reshape is bettere


In [211]:
'''kk = np.array(whole_dict_fil[2])
kk = kk.reshape(len(whole_dict_fil[2]),-1)
kk = np.mean(kk,axis=0)
# kk = np.mean(kk,axis=1)
# kk = sparse.csr_matrix(kk)
# temp = cosine_similarity(kk)
# temp = cosine_similarity(kk,dense_output=False)
np.mean(temp)'''

0.96861625

# Store Process

In [275]:
save_dir = './results/Hooklog/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df_c = df.merge(sim_df, left_index=True,right_index=True)
col = df_c.columns.tolist()
n_col = [col[12]] + col[0:12]+ col[13:]
df = df_c[n_col]
df.to_excel(saveR)
print(saveR)
df

./results/Hooklog/2020020701/losses_metrics.xlsx


,Epoch#,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num,Original_Intra_Similarity_Avg,Filter_Intra_Similarity_Avg,Original_Outra_Similarity,Filter_Outra_Similarity
0,1,2.484127,2.484127,0.585139,1.000000,0.089245,0.000000,2.483705,2.483705,0.418710,1.000000,0.163265,0.000000,0.571640,NaN,0.294336,-0.083333
1,2,2.481488,2.481488,0.574883,1.000000,0.109840,0.000000,2.482437,2.482437,0.431385,1.000000,0.163265,0.000000,0.571703,NaN,0.294415,-0.083333
2,3,2.478976,2.478976,0.559290,1.000000,0.128146,0.000000,2.481104,2.481104,0.445356,1.000000,0.163265,0.000000,0.571796,NaN,0.294588,-0.083333
3,4,2.473881,2.473881,0.547329,0.997645,0.132723,0.002355,2.479576,2.479576,0.452167,1.000000,0.163265,0.000000,0.575421,NaN,0.305074,-0.083333
4,5,2.464355,2.464355,0.542948,0.987761,0.123570,0.012435,2.475623,2.475623,0.456507,0.973344,0.040816,0.026656,0.581270,0.311835,0.321530,-0.037483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,0.039804,0.039804,0.400000,0.000000,0.990847,1.000000,0.245678,0.245678,0.600000,0.000000,0.938776,1.000000,0.546963,0.546963,0.240345,0.240345
1996,1997,0.041626,0.041626,0.400000,0.000000,0.990847,1.000000,0.245715,0.245715,0.600000,0.000000,0.938776,1.000000,0.546964,0.546964,0.240337,0.240337
1997,1998,0.044416,0.044416,0.400000,0.000000,0.990847,1.000000,0.245462,0.245462,0.600000,0.000000,0.938776,1.000000,0.546970,0.546970,0.240348,0.240348
1998,1999,0.040263,0.040263,0.400000,0.000000,0.990847,1.000000,0.245236,0.245236,0.600000,0.000000,0.938776,1.000000,0.546970,0.546970,0.240341,0.240341


In [223]:
# col = df_c.columns.tolist()
# n_col = [col[12]] + col[0:12]+ col[13:]
# df_c[n_col]

,Epoch#,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num,Original_Intra_Similarity_Avg,Filter_Intra_Similarity_Avg,Original_Outra_Similarity,Filter_Outra_Similarity
0,0,2.482105,2.482105,0.598262,0.000000,0.057208,1.000000,2.444333,2.444333,0.597253,0.000000,0.265306,1.000000,0.548994,0.548994,0.320756,3.207557e-01
1,1,2.440567,2.440567,0.597006,0.000000,0.196796,1.000000,2.388728,2.388728,0.597342,0.000000,0.346939,1.000000,0.549687,0.549687,0.317695,3.176952e-01
2,2,2.392396,2.392396,0.596791,0.000000,0.265446,1.000000,2.318130,2.318130,0.594105,0.000000,0.408163,1.000000,0.550923,0.550923,0.311467,3.114671e-01
3,3,2.322960,2.322960,0.592026,0.000000,0.272311,1.000000,2.223286,2.223286,0.590216,0.000000,0.428571,1.000000,0.552796,0.552796,0.302986,3.029862e-01
4,4,2.238850,2.238850,0.584462,0.000000,0.313501,1.000000,2.099025,2.099025,0.584394,0.000000,0.428571,1.000000,0.555114,0.555114,0.294371,2.943712e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1.996609,1.996609,0.095893,0.978485,0.274600,0.021521,1.845307,1.845307,0.132257,0.961175,0.346939,0.038942,0.554967,0.857475,0.333457,9.536743e-07
1996,1996,1.968696,1.968696,0.100493,0.977533,0.274600,0.022472,1.846353,1.846353,0.132259,0.961175,0.346939,0.038942,0.554964,0.857462,0.333433,-5.960464e-07
1997,1997,1.948684,1.948684,0.107366,0.968717,0.274600,0.031288,1.847010,1.847010,0.132204,0.961175,0.346939,0.038942,0.554970,0.857480,0.333460,2.384186e-07
1998,1998,1.976816,1.976816,0.102491,0.978300,0.274600,0.021705,1.846184,1.846184,0.132106,0.961175,0.346939,0.038942,0.554978,0.857493,0.333498,7.152557e-07


In [276]:
labelID_dict = pickle.load(open("./data/Hooklog/MikeGithub/fam_dict_enc_single_Mike.pkl","rb"))
IDlabel_dict = {v: k for k, v in labelID_dict.items()}
word_index = pickle.load(open("./data/Hooklog/MikeGithub/single_dict_enc_Mike.pkl","rb"))
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

pred_label = list(np.argmax(pred_label,axis=-1))
def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        key = item
#         print(key)
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

                 ori_word
ID                       
1          WinHttpConnect
2                 WinExec
3             OpenProcess
4           RegQueryValue
5                CopyFile
6            InternetOpen
7      WinHttpOpenRequest
8      WinHttpSendRequest
9            RegCreateKey
10           RegDeleteKey
11            LoadLibrary
12        InternetConnect
13        HttpSendRequest
14           CreateThread
15             DeleteFile
16            ExitProcess
17            WinHttpOpen
18            RegSetValue
19             CreateFile
20          CreateProcess
21           RegEnumValue
22  CreateProcessInternal
23     CreateRemoteThread


In [277]:
word_fil_text = []
word_ori_text = []
from tqdm import tqdm
for text in tqdm(int_ori_text): #text = one epoch
    temp = []
    for t in text: #t = one sample
        temp.append(getKeysByValues(list(t)))
    word_ori_text.append(temp)
for text in tqdm(int_fil_text):
    temp = []
    for t in text:
        temp.append(getKeysByValues(list(t)))
    word_fil_text.append(temp)

save_path = save_dir + "original_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_ori_text)
save_path = save_dir + "filterUsed_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_fil_text)
print(save_path)

100%|██████████| 2000/2000 [11:16<00:00,  2.96it/s]


./results/Hooklog/2020020701/filterUsed_words.pkl


In [ ]:
col = ['Epoch#','text_id','ori_text','filter_text','label','predict']
df = pd.DataFrame(columns=col)
for epoch,(text_ori,text_fil,labelID,predID) in tqdm(enumerate(zip(word_ori_text,word_fil_text,int_label,pred_label))):
    for idx,(t_o,t_f,label,pred) in enumerate(zip(text_ori,text_fil,labelID,predID)):
        temp = pd.Series([epoch+1,idx," ".join([str(x) for x in t_o])," ".join([str(x) for x in t_f]),IDlabel_dict[label],IDlabel_dict[pred]],col)
        df = df.append(temp,ignore_index=True)

save_path = save_dir + "caseStudy_result.xlsx"
df.to_excel(save_path,index=False)
print(save_path)
df

1985it [11:40,  1.29it/s]

todo: 
* 填hyperparameters
* cosine similarity比較